In [3]:
import tensorflow as tf

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [3]:
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')

In [5]:
type(x), type(y)

(tensorflow.python.ops.variables.Variable,
 tensorflow.python.ops.variables.Variable)

In [8]:
z = x*x*y + y + 2

In [9]:
type(z)

tensorflow.python.framework.ops.Tensor

**Observe:**
* 심볼변수 'z'는 단지 계산 그래프만 만들었을 뿐이다.
* 실제로 계산 그래프를 실행하려면 세션을 만들어야 한다.
  - 세션생성 : tf.Session(), 객체생성
  - 세션사용 : .run(), 그래프 값을 계산
  - 세션종료 : .close()

## 9.2 Session 실행하기

In [11]:
sess = tf.Session()
init = tf.global_variables_initializer()
# sess.run(x.initializer)
# sess.run(y.initializer)
sess.run(init)
print(sess.run(z))
sess.close()

42


In [13]:
with tf.Session() as sess:
    init.run()
    result = z.eval()
print(result)

42


In [15]:
sess = tf.InteractiveSession()
init.run()
print(z.eval())

sess.close()

42


## 9.3 계산 그래프 관리

In [16]:
reset_graph()

x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [17]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

x2.graph is graph

True

In [18]:
x2.graph is tf.get_default_graph()

False

## 9.4 노드 값의 생애주기

In [19]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

In [21]:
with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


**Note:**
* 텐서플로는 자동으로 y가 x에 의존한다는 것과 x가 w에 의존한다는 것을 감지  
  그래서 w를 평가하고 x를 그리고 y를 평가한다.
* z를 평가할때도 만찬가지이다.  
  그러나 y를 평가할때 썼던 w와 x를 재사용하지 않는다.  
  **그래서 w와 x는 2번 평가된다.** 

변수는 초기화될 때 일생이 시작되고 세션이 종료될때 마감된다.

모든 노드값은 유지되지 않는다

In [22]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


**Note:**
* w와 x를 2번 평가하지 않으려면 한번의 그래프 실행에서 y와 z를 모두 평가하도록 만듬

In [24]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()

In [25]:
theta_value

array([[-3.7185181e+01],
       [ 4.3633747e-01],
       [ 9.3952334e-03],
       [-1.0711310e-01],
       [ 6.4479220e-01],
       [-4.0338000e-06],
       [-3.7813708e-03],
       [-4.2348403e-01],
       [-4.3721911e-01]], dtype=float32)

In [26]:
# Numpy로 구하는 Theta

X = housing_data_plus_bias
y = housing.target.reshape(-1, 1)
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

print(theta_numpy)

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654266e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


In [27]:
# 사이킷런으로 구하는 Theta

from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing.data, housing.target.reshape(-1, 1))

print(np.r_[lin_reg.intercept_.reshape(-1, 1), lin_reg.coef_.T])

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


/usr/local/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


## 9.6 경사 하강법

In [28]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [29]:
print(scaled_housing_data_plus_bias.mean(axis=0))
print(scaled_housing_data_plus_bias.mean(axis=1))
print(scaled_housing_data_plus_bias.mean())
print(scaled_housing_data_plus_bias.shape)

[ 1.00000000e+00  6.60969987e-17  5.50808322e-18  6.60969987e-17
 -1.06030602e-16 -1.10161664e-17  3.44255201e-18 -1.07958431e-15
 -8.52651283e-15]
[ 0.38915536  0.36424355  0.5116157  ... -0.06612179 -0.06360587
  0.01359031]
0.11111111111111005
(20640, 9)


### 9.6.1 직접 그래디언트 계산

In [32]:
# reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name='theta')
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("에포크", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

에포크 0 MSE = 2.7544262
에포크 100 MSE = 0.632222
에포크 200 MSE = 0.5727805
에포크 300 MSE = 0.5585007
에포크 400 MSE = 0.54907
에포크 500 MSE = 0.542288
에포크 600 MSE = 0.53737885
에포크 700 MSE = 0.533822
에포크 800 MSE = 0.5312425
에포크 900 MSE = 0.5293705


In [33]:
best_theta

array([[ 2.06855226e+00],
       [ 7.74078071e-01],
       [ 1.31192386e-01],
       [-1.17845066e-01],
       [ 1.64778143e-01],
       [ 7.44081801e-04],
       [-3.91945131e-02],
       [-8.61356556e-01],
       [-8.23479712e-01]], dtype=float32)

### 9.6.2 자동미분

In [35]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name='theat')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

In [36]:
gradients = tf.gradients(mse, [theta])[0]

In [37]:
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("에포크", epoch, "MSE =", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()
    
print("best_theta:")
print(best_theta)
        

에포크 0 MSE = 2.7544262
에포크 100 MSE = 0.632222
에포크 200 MSE = 0.5727805
에포크 300 MSE = 0.5585007
에포크 400 MSE = 0.54907
에포크 500 MSE = 0.54228795
에포크 600 MSE = 0.5373789
에포크 700 MSE = 0.533822
에포크 800 MSE = 0.5312425
에포크 900 MSE = 0.5293704
best_theta:
[[ 2.06855249e+00]
 [ 7.74078071e-01]
 [ 1.31192386e-01]
 [-1.17845066e-01]
 [ 1.64778143e-01]
 [ 7.44078017e-04]
 [-3.91945094e-02]
 [-8.61356676e-01]
 [-8.23479772e-01]]


#### 임의의 함수에 대한 미분 구하기

In [1]:
def my_func(a, b):
    z = 0
    for i in range(100):
        z = a * np.cos(z + i) + z * np.sin(b - i)
    return z

In [2]:
my_func(0.2, 0.3)

-0.21253923284754916

In [5]:
a = tf.Variable(0.2, name="a")
b = tf.Variable(0.3, name="b")
z = tf.constant(0.0, name="z0")

for i in range(100):
    z = a * tf.cos(z + i) + z * tf.sin(b - i)
    
grads = tf.gradients(z, [a, b])
init = tf.global_variables_initializer()

In [6]:
with tf.Session() as sess:
    init.run()
    print(z.eval())
    print(sess.run(grads))

-0.21253741
[-1.1388494, 0.19671395]
